In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
from sklearn.metrics import f1_score
from PIL import Image
import pandas as pd

from tqdm import tqdm

In [4]:
class DeepfakeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Load real images
        real_dir = os.path.join(root_dir, 'train_images/train_images/real_train')
        for img_name in os.listdir(real_dir):
            self.images.append(os.path.join(real_dir, img_name))
            self.labels.append(0)  # 0 for real

        # Load fake images
        fake_dir = os.path.join(root_dir, 'train_images/train_images/fake_train')
        for img_name in os.listdir(fake_dir):
            self.images.append(os.path.join(fake_dir, img_name))
            self.labels.append(1)  # 1 for fake

            
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [5]:
# Define data augmentation and normalization
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
full_dataset = DeepfakeDataset(root_dir='/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv', transform=data_transforms)

# Split the dataset
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

In [7]:
# Load pre-trained VGG model
model = models.vgg16(pretrained=True)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 2)  # Binary classification

# Move model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 50
best_val_f1 = 0.0

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 227MB/s]  


In [10]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    # Validation
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())

    val_f1 = f1_score(val_labels, val_preds, average='weighted')
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), 'best_deepfake_detection_model.pth')

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Val F1: {val_f1:.4f}")


100%|██████████| 40/40 [00:10<00:00,  3.68it/s]


Epoch 1/50, Loss: 0.1006, Val F1: 0.9875


100%|██████████| 40/40 [00:10<00:00,  3.67it/s]


Epoch 2/50, Loss: 0.0585, Val F1: 0.9937


100%|██████████| 40/40 [00:10<00:00,  3.68it/s]


...

Epoch 48/50, Loss: 0.0361, Val F1: 0.9969


100%|██████████| 40/40 [00:11<00:00,  3.63it/s]


Epoch 49/50, Loss: 0.1109, Val F1: 0.9844


100%|██████████| 40/40 [00:10<00:00,  3.67it/s]


Epoch 50/50, Loss: 0.0211, Val F1: 0.9906


## Generating outputs on test set

In [30]:
class DeepfakeTest(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []

        # Load real images
        self.images_dir = os.path.join(root_dir, 'test_images/test_images/')
        for img_name in os.listdir(self.images_dir):
            self.images.append(img_name)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_name = self.images[idx]
        img_path = os.path.join(self.images_dir, image_name)
        image_id = image_name.split('.')[0].split('_')[2]
        image = Image.open(img_path).convert('RGB') 

        if self.transform:
            image = self.transform(image)

        return image, image_id


test_transforms = data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [36]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from tqdm import tqdm

# Assuming the DeepfakeTest dataset and model are defined earlier in the script

test_dataset = DeepfakeTest("/kaggle/input/wec-intelligence-sig-2024-recruitment-task-cv/", test_transforms)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

# Load best model for testing
model.load_state_dict(torch.load('best_deepfake_detection_model.pth'))
model.eval()

results = {
    'ID': [],
    'TARGET': [],
}

with torch.no_grad():
    for inputs, image_ids in tqdm(test_loader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        results['TARGET'].extend(preds.cpu().numpy())
        results['ID'].extend(image_ids)


results_df = pd.DataFrame(results)
results_df['ID'] = results_df['ID'].astype(int)
results_df = results_df.sort_values('ID')

results_df.to_csv('test_predictions.csv', index=False)

/tmp/ipykernel_30/718181326.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_deepfake_detection_model.pth'))
100%|██████████| 400/

In [39]:
output_df = pd.read_csv('test_predictions.csv')
output_df.head()

,ID,TARGET
0,1,0
1,2,0
2,3,1
3,4,0
4,5,1
